# Conversão de arquivos .zip para .csv

Os dados utilizados para o projeto são disponibilizados originalmente como diversos arquivos .txt. Estes arquivos foram baixados como dois arquivos .zip, um contendo apenas textos referentes a notícias falsas (fake.zip) e outro referente a notícias verdadeiras (true.zip). Os dois arquivos foram subidos para um bucket S3 na AWS e também estão disponíveis neste repositório, no diretório full_texts.

Para iniciar o processamento de dados, os arquivos .zip armazenados no S3 são abertos e seus conteúdos são armazenados em um DataFrame de duas colunas: 'fake' (1 se a notícia for falsa, 0 se for verdadeira) e 'text' (texto da notícia). Inicialmente são carregadas todas as notícias falsas e, em seguida, as notícias reais, de forma que as labels ficam concentradas no DataFrame, com todas as notícias falsas nos primeiros índices e todas as notícias reais concentradas nos últimos índices. Para evitar qualquer enviesamento, o DataFrame então tem suas linhas embaralhadas.

Finalmente, um arquivo csv ('dados_brutos.csv') é salvo com estes dados, para que possa ser utilizado na análise exploratória e pré-processamento de dados. Este é mantido tanto no presente repositório e uma cópia é também subida ao mesmo bucket da AWS utilizado no início do procedimento.

In [1]:
from zipfile import ZipFile
import pandas as pd
import os
import io
import s3fs
from python_scripts.save_load import BUCKET_BRUTOS, save_to_s3_bucket
fs = s3fs.S3FileSystem()

class DataKeys:
    fake = 'fake'
    text = 'text'

data_dict = {DataKeys.fake: [], DataKeys.text: []}

In [2]:
bucket_name = BUCKET_BRUTOS
bucket_path = f's3://{bucket_name}/'
fake_path = bucket_path + 'fake.zip'
true_path = bucket_path + 'true.zip'

paths_tuples = [(fake_path, 1), (true_path, 0)]

In [3]:
for s3_path, is_fake in paths_tuples:
    with fs.open(s3_path) as zip_path:
        with ZipFile(zip_path, 'r') as zip:
            for file in zip.namelist():
                if file.endswith('txt'):
                    with io.TextIOWrapper(zip.open(file), encoding="utf-8") as f:
                        file_text = f.read()
                        data_dict[DataKeys.fake].append(is_fake)
                        data_dict[DataKeys.text].append(file_text)

In [4]:
data_df = pd.DataFrame.from_dict(data_dict)
data_df = data_df.sample(frac=1).reset_index(drop=True)
data_df.head(20)

,fake,text
0,1,Revista americana publica vídeo e afirma que S...
1,0,Laudo declara que Paulo Maluf pode continuar n...
2,0,O Tribunal Regional Federal da 4.a Região (TR...
3,0,"Temer busca ponte com Toffoli, que assumirá ST..."
4,0,O lobista da Odebrecht no Congresso Claudio M...
5,0,"Terça-feira, 18 de abril de 2017. Boa noite! A..."
6,1,"Atletas chineses esculhambam o Brasil: ""Nossa ..."
7,1,Na mão grande? TCU vai fazer auditoria para ap...
8,0,Mulher presa por roubo de bebê em MG planejou ...
9,1,"Em mais um áudio, o ""doido"" xinga STF, STJ e..."


In [5]:
csv_name = 'dados_brutos.csv'
data_df.to_csv(csv_name, encoding='utf-8', index=False)
save_to_s3_bucket(csv_name, tipo='bruto')